In [126]:
### Reading all three dataframes then merging two of them to get the finbert,tf-idf and factor similarity dfs

df_1=pd.read_csv('/Users/devanshjoshi/tensorflow-test/NLP- Dan Rodriguez/Assignment_2/df_hawk-dov-tfidf')
df_1=df_1[['file_name', 'tfidf', 'hawkish', 'dovish']]
df_1 = df_1.rename(columns={'file_name': 'Filenames'})
df_2=pd.read_csv('/Users/devanshjoshi/tensorflow-test/NLP- Dan Rodriguez/Assignment_2/BERT_data.csv')
df_2=df_2[['Filename','Neutral','Negative','Positive']]
df_2 = df_2.rename(columns={'Filename': 'Filenames'})
file_path = '/Users/devanshjoshi/tensorflow-test/NLP- Dan Rodriguez/Assignment_2/FOMC_Data_2011_2024.xlsx'
df_dict = pd.read_excel(file_path, sheet_name=None, parse_dates=True, index_col=0)
df= pd.merge(df_1, df_2, on='Filenames', how='inner')
df['dates'] = df['Filenames'].str.extract(r'(\d{8})')
df

,Filenames,tfidf,hawkish,dovish,Neutral,Negative,Positive,dates
0,FOMCminutes20131218.txt,20.529626,0.130615,0.537401,0.842367,0.015655,0.141978,20131218
1,FOMCminutes20121023.txt,16.736768,0.137413,0.547877,0.773134,0.040459,0.186407,20121023
2,FOMCminutes20181108.txt,13.417258,0.137908,0.538247,0.752988,0.243545,0.003467,20181108
3,FOMCminutes20231101.txt,14.167732,0.135363,0.535311,0.544957,0.432212,0.022831,20231101
4,FOMCminutes20161214.txt,12.337477,0.137458,0.539062,0.873217,0.092973,0.033810,20161214
...,...,...,...,...,...,...,...,...
912,brainard20141202a.txt,4.514435,0.159401,0.535447,0.999678,0.000174,0.000148,20141202
913,powell20180620a.txt,8.119419,0.319314,0.644811,0.669202,0.312838,0.017960,20180620
914,waller20230511a.txt,17.852031,0.136195,0.521566,0.975480,0.001239,0.023281,20230511
915,powell20190509a.txt,2.168036,0.158238,0.548112,0.998713,0.000368,0.000918,20190509


In [127]:
import pandas as pd

def calculate_one_day_returns(input_date, df_dict): ## calculate return over a period of one day
    prev_date = pd.to_datetime(input_date, format='%Y%m%d') - pd.Timedelta(days=1)
    input_date = pd.to_datetime(input_date, format='%Y%m%d') 
    next_date = pd.to_datetime(input_date, format='%Y%m%d') + pd.Timedelta(days=1)
    
    # Skip weekends (Saturday, Sunday)
    while prev_date.weekday() >= 5:  # 5 = Saturday, 6 = Sunday
        prev_date -= pd.Timedelta(days=1)
    
    while next_date.weekday() >= 5:  # 5 = Saturday, 6 = Sunday
        next_date += pd.Timedelta(days=1)
    
    returns = {}
    
    for sheet_name, df in df_dict.items(): ## iterate over df of dictionaries
        sheet_name = sheet_name.strip()
        try:
            if sheet_name in ['GT10', 'GT2']:
                price_today = df.loc[input_date]['PX_MID'] ##calculate todays close price
                price_prev_date = df.loc[prev_date]['PX_MID'] ##calculate prev close price
                price_next_date = df.loc[next_date]['PX_MID'] ##calculate next close price
            else:
                price_today = df.loc[input_date]['PX_LAST'] ##calculate todays close price
                price_prev_date = df.loc[prev_date]['PX_LAST'] ##calculate prev close price
                price_next_date = df.loc[next_date]['PX_LAST'] ##calculate next close price

            # Calculate 1-day return
            return_ = (price_today-price_prev_date) / price_prev_date
#             return_ = (price_next_date-price_today) / price_today
            returns[sheet_name] = return_

        except KeyError as e:
            pass

    return returns

# Wrapper function to use in df.apply() that calls calculate_one_day_returns
def apply_one_day_returns(input_date, df_dict):
    return calculate_one_day_returns(input_date, df_dict)

# Apply the function to each date in the 'dates' column using df.apply()
df['returns'] = df['dates'].apply(lambda x: apply_one_day_returns(x, df_dict))

# Convert the 'returns' column into separate columns for each asset (if needed)
# This assumes that 'returns' contains a dictionary for each row
returns_df = pd.DataFrame(df['returns'].tolist())

# Concatenate the original DataFrame and the new returns DataFrame
df = pd.concat([df, returns_df], axis=1)

# Drop the 'returns' column if it's no longer needed
df.drop(columns=['returns'], inplace=True)

# Sort the DataFrame by the 'dates' column
df['dates'] = pd.to_datetime(df['dates'], format='%Y%m%d')  # Ensure 'dates' is in datetime format
df = df.sort_values(by='dates')

df.dropna(inplace=True)
if 'dates' in df.columns:
    df.set_index('dates',inplace=True)
df


,Filenames,tfidf,hawkish,dovish,Neutral,Negative,Positive,GT10,GT2,2s10s_Spread,Gold_Prices,VIX,SP500
dates,,,,,,,,,,,,,
2012-01-06,duke20120106a.txt,34.772229,0.121506,0.586030,0.608239,0.036264,0.355497,-0.018537,-0.011494,-0.019250,-0.003470,-0.039572,-0.002537
2012-01-06,raskin20120106a.txt,18.267792,0.144345,0.564113,0.745609,0.001359,0.253032,-0.018537,-0.011494,-0.019250,-0.003470,-0.039572,-0.002537
2012-01-25,FOMCminutes20120125.txt,29.661542,0.131188,0.545940,0.883420,0.012123,0.104456,-0.032023,-0.050633,-0.029261,0.027353,-0.031729,0.008664
2012-01-25,FOMCpresconf20120125.txt,22.602942,0.207025,0.684904,0.701371,0.007351,0.291277,-0.032023,-0.050633,-0.029261,0.027353,-0.031729,0.008664
2012-02-16,bernanke20120216a.txt,14.048838,0.130207,0.535245,0.918289,0.015581,0.066130,0.029046,0.106061,0.016249,0.000041,-0.090823,0.011026
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-09-26,cook20240926a.txt,9.342633,0.113002,0.548505,0.851006,0.021251,0.127743,0.002905,0.019663,-0.257553,0.005830,-0.002596,0.004039
2024-09-30,powell20240930a.txt,2.100760,0.357559,0.602653,0.466771,0.149609,0.383620,0.007996,0.023034,-0.273074,-0.008901,-0.013561,0.004237
2024-09-30,bowman20240930a.txt,10.567125,0.247093,0.597960,0.161292,0.498050,0.340658,0.007996,0.023034,-0.273074,-0.008901,-0.013561,0.004237


In [142]:
import pandas as pd
import matplotlib.pyplot as plt


columns_to_quintile = ['tfidf', 'hawkish', 'dovish','Negative', 'Positive'] ##column to divide quintiles based on
columns_to_average = ['GT10', 'GT2', '2s10s_Spread', 'Gold_Prices', 'VIX', 'SP500']  ##column to calculated median based on

# Function to divide a column into quintiles and calculate the median of related columns
def calculate_quintile_means(df, column, columns_to_average):
    # Create quintile bins (1-5)
    df['quintile'] = pd.qcut(df[column], q=5, labels=False) + 1

    # Group by quintiles and calculate the median of the target columns, do not go by the name of the variable
    quintile_means = df.groupby('quintile')[columns_to_average].median()
    
    return quintile_means ##do not go by the name of the variable

# Dictionary to hold quintile medians for each column ##do not go by the name of the variable
quintile_means_dict = {}

# Calculate quintile medians for each column in columns_to_quintile
for column in columns_to_quintile:
    quintile_means = calculate_quintile_means(df, column, columns_to_average)
    quintile_means_dict[column] = quintile_means
    
for column in columns_to_quintile:
    plt.figure(figsize=(12,10))
    quintile_means_dict[column].plot(kind='line', marker='o')
    plt.title(f'Quintile Median for {column}')
    plt.xlabel('Quintile')
    plt.ylabel('Median')
    plt.tight_layout()
    file_name = f'quintile_median_{column}.png'
    plt.savefig(f'/Users/devanshjoshi/tensorflow-test/NLP- Dan Rodriguez/Assignment_2/{file_name}')
    plt.close()

# # Plotting
# fig, axes = plt.subplots(nrows=len(columns_to_quintile), ncols=1, figsize=(10, 30))

# for i, column in enumerate(columns_to_quintile):
#     quintile_means_dict[column].plot(kind='line', ax=axes[i], marker='o')  # Use line chart with markers
#     axes[i].set_title(f'Quintile Median for {column}')
#     axes[i].set_xlabel('Quintile')
#     axes[i].set_ylabel('Median')

# plt.tight_layout()
# plt.show()


<Figure size 864x720 with 0 Axes>

<Figure size 864x720 with 0 Axes>

<Figure size 864x720 with 0 Axes>

<Figure size 864x720 with 0 Axes>

<Figure size 864x720 with 0 Axes>

In [129]:
import pandas as pd
import statsmodels.api as sm

# List of dependent variables
dependent_vars = ['GT10', 'GT2', '2s10s_Spread', 'Gold_Prices', 'VIX', 'SP500']

# Independent variables grouped
group_1_indep_vars = ['tfidf']
group_2_indep_vars = ['hawkish', 'dovish']
group_3_indep_vars = ['Negative', 'Positive']

# Function to perform multiple regression and return R², p-values, and coefficients
def perform_regression(df, independent_vars, dependent_var):
    X = df[independent_vars]  # Multiple independent variables (predictors)
    y = df[dependent_var]  # Dependent variable (response)
    
    # Add constant to the independent variables (intercept term)
    X = sm.add_constant(X)
    
    # Perform OLS regression
    model = sm.OLS(y, X).fit()
    
    # Get R², p-values, and coefficients
    r_squared = model.rsquared
    p_values = model.pvalues
    coefficients = model.params
    
    return r_squared, p_values, coefficients

# List to store the results for DataFrame creation
results_list = []

# Perform regression for group_1 ('tfidf' alone)
for dep_var in dependent_vars:
    r_squared, p_values, coefficients = perform_regression(df, group_1_indep_vars, dep_var)
    
    # Append the results to the list
    for var in group_1_indep_vars + ['const']:
        results_list.append({'Dependent_Var': dep_var,
                             'Independent_Var': var,
                             'R²': r_squared,
                             'Coefficient': coefficients[var],
                             'P-value': p_values[var]})

# Perform regression for group_2 ('hawkish', 'dovish' together)
for dep_var in dependent_vars:
    r_squared, p_values, coefficients = perform_regression(df, group_2_indep_vars, dep_var)
    
    # Append the results to the list
    for var in group_2_indep_vars + ['const']:
        results_list.append({'Dependent_Var': dep_var,
                             'Independent_Var': var,
                             'R²': r_squared,
                             'Coefficient': coefficients[var],
                             'P-value': p_values[var]})

# Perform regression for group_3 ('Neutral', 'Negative', 'Positive' together)
for dep_var in dependent_vars:
    r_squared, p_values, coefficients = perform_regression(df, group_3_indep_vars, dep_var)
    
    # Append the results to the list
    for var in group_3_indep_vars + ['const']:
        results_list.append({'Dependent_Var': dep_var,
                             'Independent_Var': var,
                             'R²': r_squared,
                             'Coefficient': coefficients[var],
                             'P-value': p_values[var]})

# Convert the results list to a DataFrame
results_df = pd.DataFrame(results_list)

# Display the resulting DataFrame
print(results_df)


   Dependent_Var Independent_Var        R²   Coefficient   P-value
0           GT10           tfidf  0.000500 -7.147532e-05  0.514798
1           GT10           const  0.000500  3.775783e-04  0.819217
2            GT2           tfidf  0.000554 -1.459023e-04  0.492768
3            GT2           const  0.000554  3.193375e-03  0.318827
4   2s10s_Spread           tfidf  0.000060 -2.959039e-04  0.821345
5   2s10s_Spread           const  0.000060  8.340956e-03  0.672501
6    Gold_Prices           tfidf  0.001875  5.346232e-05  0.207015
7    Gold_Prices           const  0.001875  9.153706e-05  0.885858
8            VIX           tfidf  0.007057 -8.277396e-04  0.014231
9            VIX           const  0.007057  6.557516e-03  0.196565
10         SP500           tfidf  0.002549  5.966276e-05  0.141151
11         SP500           const  0.002549  2.811282e-04  0.644978
12          GT10         hawkish  0.005368 -1.286362e-02  0.217985
13          GT10          dovish  0.005368 -1.659283e-02  0.37